In [18]:
!rm *

rm: cannot remove 'sample_data': Is a directory


# RNN Based molucule generation 

Laurent Cetinsoy

In the hands-on we will train a character based RNN to generate smile molecules

We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model ! 

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [19]:
!wget https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv

--2022-04-02 16:39:11--  https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22606589 (22M) [text/plain]
Saving to: ‘250k_smiles.csv’

250k_smiles.csv     100%[===================>]  21.56M  --.-KB/s    in 0.1s    

2022-04-02 16:39:11 (226 MB/s) - ‘250k_smiles.csv’ saved [22606589/22606589]



Import pandas and load the first 1000 lines

In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv("250k_smiles.csv")
df.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


Display the first rows of the dataframe

In [21]:
print(df.iloc[0])

smiles    CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n
logP                                         5.0506
qed                                        0.702012
SAS                                        2.084095
Name: 0, dtype: object


## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [22]:
print("The bigest smile molecule have a size f : ", df["smiles"].str.len().max())

The bigest smile molecule have a size f :  110



Code a function **unic_characters(string)** which return the unic characters in a string


In [23]:
def unic_characters(string):
    return list(set(string))

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [24]:
unic_characters_in_list = unic_characters(df["smiles"].str.cat())

print(unic_characters_in_list)

['s', '7', 'I', '6', '5', ']', '\n', 'r', '@', 'o', 'O', '-', '(', '2', '\\', '+', '[', '3', 'n', '=', ')', 'H', '8', 'c', 'l', 'B', 'C', 'S', '/', '1', 'P', 'F', '#', 'N', '4']


Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")


In [29]:
def map_char_to_int(unic_chars):
    return dict([(unic_character, ord(unic_character)) for unic_character in unic_characters_in_list])

dict_char_to_int = map_char_to_int(unic_characters_in_list)

print(dict_char_to_int)

{'s': 115, '7': 55, 'I': 73, '6': 54, '5': 53, ']': 93, '\n': 10, 'r': 114, '@': 64, 'o': 111, 'O': 79, '-': 45, '(': 40, '2': 50, '\\': 92, '+': 43, '[': 91, '3': 51, 'n': 110, '=': 61, ')': 41, 'H': 72, '8': 56, 'c': 99, 'l': 108, 'B': 66, 'C': 67, 'S': 83, '/': 47, '1': 49, 'P': 80, 'F': 70, '#': 35, 'N': 78, '4': 52}


Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [30]:
def map_int_to_char(unic_chars):
    return dict([(ord(unic_character), unic_character) for unic_character in unic_characters_in_list])

dict_int_to_char = map_int_to_char(unic_characters_in_list)

print(dict_int_to_char)

{115: 's', 55: '7', 73: 'I', 54: '6', 53: '5', 93: ']', 10: '\n', 114: 'r', 64: '@', 111: 'o', 79: 'O', 45: '-', 40: '(', 50: '2', 92: '\\', 43: '+', 91: '[', 51: '3', 110: 'n', 61: '=', 41: ')', 72: 'H', 56: '8', 99: 'c', 108: 'l', 66: 'B', 67: 'C', 83: 'S', 47: '/', 49: '1', 80: 'P', 70: 'F', 35: '#', 78: 'N', 52: '4'}


For each smile molecule add the ending token to it

We have a "\n" at the end of each molecule so we do not need to add it

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

n_feature is a representation of the dimanesion of our data that is to say that we can have our data in very big dimension and so in with a big n_feature

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

In [43]:
def build_X_and_y(smile, i_char, seq_length):

    X = [smile[i: i + seq_length] for i in i_char]
    y = [smile[i + seq_length] for i in i_char]
    
    return X, y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [44]:
build_X_and_y("C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n", [1,2,3], 4)

(['[C@@', 'C@@H', '@@H]'], ['H', ']', '1'])

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

Create numpy arrays from the lists

Reshape the X numpy array (n_examples, seq_lenght, 1)

Normalize X by dividing each values by the total number of unic characters

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


Compile the model with the appropriate loss function and the adam optimizer

Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : predict the molecule solubility of your generated molecules 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better